In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import gc
os.getcwd()

# To check about the 2019 weekly file path df week filter for the rolling later

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration'

In [2]:
recent_complete_quarter_End=datetime.date(2018,5,5)
prior_quarter_End=recent_complete_quarter_End-datetime.timedelta(days=7*13)


print("prior_quarter_End",prior_quarter_End)
print("recent_complete_quarter_End",recent_complete_quarter_End)

lapsed_date_begin=prior_quarter_End-datetime.timedelta(days=(4*365+1+6))
print("lapsed_date_begin",lapsed_date_begin)


prior_quarter_End 2018-02-03
recent_complete_quarter_End 2018-05-05
lapsed_date_begin 2014-01-28


In [3]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()

print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_date_begin)]
lapsed=lapsed[lapsed['transaction_date']<=str(recent_complete_quarter_End)]

print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

2014-08-26
2017-02-26
2014-08-26
2017-02-26


In [4]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_2018Q1=pd.DataFrame()

count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    
    chunk=chunk[(chunk['transaction_date']>=str(lapsed_date_begin)) & (chunk['transaction_date']<=str(prior_quarter_End))]
    chunk = chunk.drop_duplicates()
    
    dftrans_before_2018Q1=dftrans_before_2018Q1.append(chunk)
    
    count_i+=1
    print(count_i,datetime.datetime.now())
dftrans_before_2018Q1=dftrans_before_2018Q1.drop_duplicates()   


1 2019-05-28 14:13:52.971537
2 2019-05-28 14:14:12.277591
3 2019-05-28 14:14:29.597348
4 2019-05-28 14:14:48.608094
5 2019-05-28 14:15:19.035816
6 2019-05-28 14:15:50.124336
7 2019-05-28 14:16:21.396907
8 2019-05-28 14:16:53.481296
9 2019-05-28 14:17:25.867484
10 2019-05-28 14:18:00.359558
11 2019-05-28 14:18:31.456424
12 2019-05-28 14:19:20.352137
13 2019-05-28 14:22:16.614735
14 2019-05-28 14:23:47.095059
15 2019-05-28 14:24:13.112484


In [5]:
all_rewards_most_recent=dftrans_before_2018Q1.copy()
del chunk
del dftrans_before_2018Q1
gc.collect()

# Already filterd of the dates above
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
print(all_rewards_most_recent.shape)



print(all_rewards_most_recent['transaction_date'].min())
print(all_rewards_most_recent['transaction_date'].max())

(18422325, 2)
2016-06-26
2018-02-03


In [6]:
all_rewards_most_recent=all_rewards_most_recent.append(lapsed)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])


print(all_rewards_most_recent['transaction_date'].min())
print(all_rewards_most_recent['transaction_date'].max())

2014-08-26
2018-02-03


In [7]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)



In [8]:
df_all_daily_files_in_recent_quarter=list(recursive_file_gen('/home/jian/BigLots/hist_daily_data_subclasslevel/'))
df_all_daily_files_in_recent_quarter=[x for x in df_all_daily_files_in_recent_quarter if ".txt" in x]
df_all_daily_files_in_recent_quarter=pd.DataFrame({"file_path":df_all_daily_files_in_recent_quarter})
df_all_daily_files_in_recent_quarter['week_end_dt']=df_all_daily_files_in_recent_quarter['file_path'].apply(lambda x: x.split("l/MediaStormDailySales_week_ending_")[1][:10])
df_all_daily_files_in_recent_quarter=df_all_daily_files_in_recent_quarter[df_all_daily_files_in_recent_quarter['week_end_dt']>str(prior_quarter_End)]
df_all_daily_files_in_recent_quarter=df_all_daily_files_in_recent_quarter[df_all_daily_files_in_recent_quarter['week_end_dt']<=str(recent_complete_quarter_End)]
df_all_daily_files_in_recent_quarter.shape


(13, 2)

In [9]:
RecentQuarter_sales_by_id=pd.DataFrame()
dftrans_only_2018Q1=pd.DataFrame()
for file in df_all_daily_files_in_recent_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','transaction_dt','subclass_transaction_amt'],dtype=str)
    df=df[~pd.isnull(df['customer_id_hashed'])]
    
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['customer_id_hashed'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    df=df[['customer_id_hashed','transaction_dt']].drop_duplicates()
    RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.append(df_sales)
    dftrans_only_2018Q1=dftrans_only_2018Q1.append(df)
    print(file,datetime.datetime.now())
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.groupby("customer_id_hashed")['subclass_transaction_amt'].sum().to_frame().reset_index()
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.rename(columns={"subclass_transaction_amt":"sales_recent_Quarter"})
RecentQuarter_sales_by_id['Recent_Quarter_Shopping_Group']="Recent_Quarter_Shopped"

dftrans_only_2018Q1=dftrans_only_2018Q1.sort_values(["customer_id_hashed","transaction_dt"],ascending=[True,False])
dftrans_only_2018Q1=dftrans_only_2018Q1.drop_duplicates("customer_id_hashed")


/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-03-31.txt 2019-05-28 14:56:01.246008
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-02-17.txt 2019-05-28 14:56:29.971026
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-02-24.txt 2019-05-28 14:56:59.320546
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-03-03.txt 2019-05-28 14:57:32.414935
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-02-10.txt 2019-05-28 14:57:57.990729
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-04-07.txt 2019-05-28 14:58:37.055017
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-04-21.txt 2019-05-28 14:59:06.354246
/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-03-17.txt 2019-05-28 14:59:38.477060
/home/ji

In [10]:
dftrans_only_2018Q1=dftrans_only_2018Q1.rename(columns={"transaction_dt":"transaction_date"})

print(dftrans_only_2018Q1['transaction_date'].min())
print(dftrans_only_2018Q1['transaction_date'].max())


2018-02-04
2018-05-05


In [11]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date']

all_rewards_most_recent_After=all_rewards_most_recent[['customer_id_hashed','transaction_date']].append(dftrans_only_2018Q1)
all_rewards_most_recent_After=all_rewards_most_recent_After.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_the_Quarter"})

all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After

gc.collect()

79

In [12]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_before_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_before_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_pre_Quarter_End']=prior_quarter_End-all_rewards_most_recent['transaction_date_before_the_Quarter']
# Filled the na before with a later adte 2019-12-31


all_rewards_most_recent['transaction_date_after_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_after_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_recentt_Quarter_End']=recent_complete_quarter_End-all_rewards_most_recent['transaction_date_after_the_Quarter']


In [13]:
all_rewards_most_recent['Month_to_pre_Quarter_End']=all_rewards_most_recent['Days_to_pre_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent['Month_to_recent_Quarter_End']=all_rewards_most_recent['Days_to_recentt_Quarter_End'].apply(lambda x: x.days/(365.25/12))


In [14]:
all_rewards_most_recent['Group_before_the_Quarter']=np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>12) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']<0,"NotAvailable_Before_Quarter","NaN")
                                                                    )
                                                           )
                                                   )
all_rewards_most_recent['Group_after_the_Quarter']=np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>12) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                    )
                                                           )
                                                   )

In [15]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,RecentQuarter_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent.shape

(20923698, 12)

In [17]:
all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("Recent_Quarter_No_Shop")

In [ ]:
###########

In [25]:
all_rewards_most_recent.columns.tolist()

['customer_id_hashed',
 'transaction_date',
 'transaction_date_before_the_Quarter',
 'transaction_date_after_the_Quarter',
 'Days_to_pre_Quarter_End',
 'Days_to_recentt_Quarter_End',
 'Month_to_pre_Quarter_End',
 'Month_to_recent_Quarter_End',
 'Group_before_the_Quarter',
 'Group_after_the_Quarter',
 'sales_recent_Quarter',
 'Recent_Quarter_Shopping_Group',
 'sign_up_date',
 'NewRewards']

In [19]:
output_id_count=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_Q4_sales=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group'])['sales_recent_Quarter'].sum().to_frame().reset_index()

output=pd.merge(output_id_count,output_id_Q4_sales,on=['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group'],how="outer")
output


,Group_before_the_Quarter,Group_after_the_Quarter,Recent_Quarter_Shopping_Group,id_count,sales_recent_Quarter
0,Active,Active,Recent_Quarter_No_Shop,7921269,0.000000e+00
1,Active,Active,Recent_Quarter_Shopped,5436771,4.702312e+08
2,Active,Lapsed_13_48,Recent_Quarter_No_Shop,1531848,0.000000e+00
3,Lapsed_13_48,Active,Recent_Quarter_Shopped,370771,2.798032e+07
4,Lapsed_13_48,Lapsed_13_48,Recent_Quarter_No_Shop,4120398,0.000000e+00
5,NotAvailable_Before_Quarter,Active,Recent_Quarter_Shopped,1542641,1.738370e+08


In [20]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration'

In [21]:
output['Recent_Quarteer_Sale_per_ID']=output['sales_recent_Quarter']/output['id_count']
output['Recent_Quarteer_Sale_per_ID']=output['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))



In [22]:
output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2018Q1_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [23]:
all_rewards_most_recent.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2018Q1_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [24]:
all_rewards_most_recent.shape

(20923698, 12)

In [25]:
len(all_rewards_most_recent['customer_id_hashed'].unique())

20923698